In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 0.4 범위내 추출

In [ ]:
import pandas as pd
import os

# 경로 설정
directory_path = '/content/drive/MyDrive/Kwargs/esg관련도/test'

# 0으로 라벨링할 데이터프레임을 초기화
selected_data = pd.DataFrame()

# 모든 CSV 파일에 대해 처리
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path)

        # 특정 조건에 맞는 데이터 선택
        filtered_df = df[(df['sentiment_score'] >= -0.4) & (df['sentiment_score'] <= 0.4)]



        # 라벨 0으로 설정
        filtered_df['label'] = 0

        # 필요한 컬럼만 남기고, full_text를 text로 이름 변경
        filtered_df = filtered_df[['full_text', 'label']].rename(columns={'full_text': 'text'})

        # 선택된 데이터프레임에 추가
        selected_data = pd.concat([selected_data, filtered_df], ignore_index=True)


selected_data.sample()


<ipython-input-6-f667d910dea6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['label'] = 0
<ipython-input-6-f667d910dea6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['label'] = 0
<ipython-input-6-f667d910dea6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

,text,label
954,한진 때와 판박이불 붙는 금호석화 표대결 머니투데이 김성은 기자 월 주주총회를 앞두...,0


### 무관데이터 추출

In [ ]:
import os
import json
import pandas as pd
from tqdm import tqdm

# 파일 경로 및 파일명 리스트
base_path = '/content/drive/MyDrive/Kwargs/무관데이터/kakao_1'  # Colab에서는 '/content/'를 사용. 필요 시 수정
save_path =  '/content/drive/MyDrive/Kwargs/무관데이터'
file_prefix = 'KAKAO_'
file_suffix = '.json'

# 데이터 저장을 위한 리스트
data = []

# 파일 범위 설정
prefix_range_start = 899
prefix_range_end = 1798
suffix_range_start = 1
suffix_range_end = 14

# tqdm을 사용하여 진행 상황 표시
total_files = (prefix_range_end - prefix_range_start + 1) * (suffix_range_end - suffix_range_start + 1)
progress_bar = tqdm(total=total_files, desc="Processing files")

# 각 파일을 순회하며 데이터 추출
for prefix in range(prefix_range_start, prefix_range_end + 1):
    for suffix in range(suffix_range_start, suffix_range_end + 1):
        file_name = f"{file_prefix}{prefix}_{str(suffix).zfill(2)}{file_suffix}"
        file_path = os.path.join(base_path, file_name)

        if not os.path.exists(file_path):
            print(f"파일을 찾을 수 없습니다: {file_path}")
            progress_bar.update(1)
            continue

        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = json.load(file)
                if 'info' in content:
                    for item in content['info']:
                        norm_texts = [line['norm_text'] for line in item['annotations']['lines']]
                        combined_text = ' '.join(norm_texts)
                        data.append({'filename': file_name, 'norm_text': combined_text})
        except json.JSONDecodeError:
            print(f"JSONDecodeError: 파일을 읽는 중 오류가 발생했습니다: {file_path}")
        except Exception as e:
            print(f"예상치 못한 오류가 발생했습니다: {file_path}, 오류: {str(e)}")

        # 진행 상황 업데이트
        progress_bar.update(1)

# 데이터프레임 생성
df = pd.DataFrame(data)

# CSV로 저장
output_file = os.path.join(save_path, 'combined_norm_texts.csv')
df.to_csv(output_file, index=False, encoding='utf-8-sig')

# tqdm 종료
progress_bar.close()

print(f"CSV 파일로 저장되었습니다: {output_file}")



Processing files:   0%|          | 63/12600 [01:43<5:43:14,  1.64s/it]

Processing files:   1%|▏         | 159/12600 [00:50<1:51:56,  1.85it/s]

KeyboardInterrupt: 

### 부정변환 텍스트

In [ ]:
import pandas as pd

def extract_and_save_sentences(input_file, output_file):
    sentences = []
    with open(input_file, 'r', encoding='utf-8') as file:
        for line in file:
            if line.startswith("Back to Korean:"):
                sentence = line.split("Back to Korean:")[1].strip()
                sentences.append(sentence)

    df = pd.DataFrame(sentences, columns=["Back to Korean Sentences"])
    df.to_csv(output_file, index=False, encoding='utf-8')

# Usage
input_file = '/content/drive/MyDrive/Colab Notebooks/부정 텍스트 변환 0710.txt'
output_file = '/content/drive/MyDrive/Kwargs/부정 텍스트 변환 0710.csv'
extract_and_save_sentences(input_file, output_file)


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Kwargs/부정 텍스트 변환 0710.csv')
df

,Back to Korean Sentences
0,조직 내부의 ESG위원회를 설치하거나 외부 전문가를 참여시켜 주주의 의사에 부합하고...
1,ESG 관점에서 모듈형 건설 방식은 혁신적이고 지속 가능하지만 공장에서 현장까지 건...
2,ESG 관점에서 베트남(비나)은 2021년 평가에서 긍정적인 성과를 거두기 위해 인...
3,근로 기본권을 지키지 아니한다 □ 지방법에 따라 근로자의 근로 기본권을 존중하지 못한다.
4,"ESG 관점에서 사외이사는 SEC 규정 4200이 허용되지 않는 한 ""현 회계연도 ..."
...,...
5116,ESG 관점에서 3단계 전략적 이슈 선택은 다양한 성과 이슈를 효과적으로 다루지 못...
5117,"나아가, 우리는 ESG 관점에서 스범위 3 배출량을 측정하고 완화하기 위해 공급 사..."
5118,ESG 관점에서 부패에 대한 관용을 허용하지 않고 책임 집행과 관련하여 개인 이익 ...
5119,ESG 관점에서 SK스퀘어와의 분할은 비핵심 ICT 사업 역량을 소홀히 하고 사업 ...


### 3종세트 등장

In [ ]:
import pandas as pd
import os
from tqdm import tqdm

# 파일 경로
#base_path_label_0 = '/content/drive/MyDrive/Kwargs/무관데이터/combined_norm_texts.csv'
df_label_0 = selected_data
base_path_label_1 = '/content/drive/MyDrive/Kwargs/esg보고서/라벨링/'
#base_path_label_minus_1 = '/content/drive/MyDrive/Kwargs/부정 텍스트 변환 0710.csv'
base_path_label_minus_1 = '/content/drive/MyDrive/Kwargs/Generated_Combinations.csv'

# 파일 리스트 가져오기
files_label_1 = [os.path.join(base_path_label_1, f) for f in os.listdir(base_path_label_1) if f.endswith('.csv')]

In [ ]:
# 라벨 0 데이터 읽기
df_label_0 = pd.read_csv(base_path_label_0)
df_label_0['label'] = 0
df_label_0.rename(columns={'norm_text': 'text'}, inplace=True)

In [ ]:
# 라벨 1 데이터 읽기
data_label_1 = []
for file in tqdm(files_label_1, desc="Processing label 1 files"):
    df = pd.read_csv(file)
    df['label'] = 1
    df.rename(columns={'sentence': 'text'}, inplace=True)
    data_label_1.append(df)
df_label_1 = pd.concat(data_label_1, ignore_index=True)

Processing label 1 files: 100%|██████████| 150/150 [00:05<00:00, 28.13it/s]


In [ ]:
# 라벨 -1 데이터 읽기
df_label_minus_1 = pd.read_csv(base_path_label_minus_1)
df_label_minus_1['label'] = -1
df_label_minus_1.rename(columns={'norm_text': 'text'}, inplace=True)
df_label_minus_1.rename(columns={'Back to Korean Sentences': 'text'}, inplace=True)

In [ ]:
df_label_1.rename(columns={'문장': 'text'}, inplace=True)

### 데이터 불균형 해소

In [ ]:
import random
from eda import eda
import pandas as pd

# 데이터 불러오기
data = df_label_0

# 라벨이 적은 클래스의 데이터 선택
minority_class = data[data['label'] == data['label'].value_counts().idxmin()]

# 증강할 데이터 수
augmentation_factor = 1.7
num_augment = int(len(minority_class) * (augmentation_factor - 1))

augmented_texts = []
augmented_labels = []

for i in range(num_augment):
    original_text = random.choice(minority_class['text'].values)
    augmented_text = eda(original_text, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=1)
    augmented_texts.append(augmented_text[0])
    augmented_labels.append(minority_class['label'].values[0])

# 증강된 데이터프레임 생성
augmented_data = pd.DataFrame({'text': augmented_texts, 'label': augmented_labels})

# 원본 데이터에 증강된 데이터 추가
balanced_data = pd.concat([data, augmented_data], ignore_index=True)

# 결과를 CSV로 저장
df_label_0 = balanced_data

ModuleNotFoundError: No module named 'eda'

### 데이터 증강

In [ ]:
# 라벨이 0인 데이터를 3배로 증강
label_0_data = df_label_0[df_label_0['label'] == 0]
df_label_0 = pd.concat([df_label_0, label_0_data, label_0_data], ignore_index=True)

### 본격 합치기 시작

In [ ]:
import pandas as pd

# 데이터 이어붙이기 함수
def combine_texts(df, num_texts):
    combined_texts = []
    df['text'].fillna('', inplace=True)  # NaN 값을 빈 문자열로 대체
    for i in range(0, len(df), num_texts):
        combined_text = ' '.join(df['text'][i:i+num_texts])
        combined_texts.append(combined_text)
    return pd.DataFrame(combined_texts, columns=['text'])

In [ ]:
# 라벨 0: 3개씩 이어붙이기
df_label_0_combined = combine_texts(df_label_0, 1)
df_label_0_combined['label'] = 0

# 라벨 1: 15개씩 이어붙이기
df_label_1_combined = combine_texts(df_label_1, 15)
df_label_1_combined['label'] = 1

# 라벨 -1: 그대로 사용
df_label_m1_combined = df_label_minus_1[['text', 'label']]

In [ ]:

# 모든 데이터 통합
df_combined = pd.concat([df_label_0_combined, df_label_1_combined, df_label_m1_combined], ignore_index=True)

# NaN 값을 빈 문자열로 대체
df_combined['text'].fillna('', inplace=True)

# 라벨별 텍스트 행의 평균 길이와 갯수 계산
df_combined['text_length'] = df_combined['text'].apply(len)
label_stats = df_combined.groupby('label').agg({'text_length': 'mean', 'text': 'count'}).rename(columns={'text_length': 'avg_text_length', 'text': 'count'})

# 통합된 데이터 저장
output_file = '/content/drive/MyDrive/Kwargs/combined_labeled_data_balanced_0716.csv'
df_combined.to_csv(output_file, index=False, encoding='utf-8-sig')

# 결과 출력
print(f"CSV 파일로 저장되었습니다: {output_file}")
print(label_stats)


CSV 파일로 저장되었습니다: /content/drive/MyDrive/Kwargs/combined_labeled_data_balanced_0716.csv
       avg_text_length  count
label                        
-1         2145.809000   5000
 0         1466.760204   5292
 1         2288.836983  19360


In [ ]:
# 라벨 별로 가장 작은 데이터 수에 맞추어 undersampling
min_count = df_combined['label'].value_counts().min()


df_label_0_final = df_label_0_combined.sample(n=min_count, random_state=42)
df_label_1_final = df_label_1_combined.sample(n=min_count, random_state=42)
df_label_minus_1_final = df_label_m1_combined.sample(n=min_count, random_state=42)

# 최종 데이터 통합
df_final = pd.concat([df_label_0_final, df_label_1_final, df_label_minus_1_final], ignore_index=True)

# 라벨별 텍스트 행의 평균 길이와 갯수 계산
df_final['text_length'] = df_final['text'].apply(len)
label_stats_final = df_final.groupby('label').agg({'text_length': 'mean', 'text': 'count'}).rename(columns={'text_length': 'avg_text_length', 'text': 'count'})

# 통합된 데이터 저장
output_file_final = '/content/drive/MyDrive/Kwargs/combined_labeled_data_balanced_0719.csv'
df_final.to_csv(output_file_final, index=False, encoding='utf-8-sig')

# 결과 출력
print(f"CSV 파일로 저장되었습니다: {output_file_final}")
print(label_stats_final)

CSV 파일로 저장되었습니다: /content/drive/MyDrive/Kwargs/combined_labeled_data_balanced_0719.csv
       avg_text_length  count
label                        
-1           2145.8090   5000
 0           1467.5248   5000
 1           2315.9126   5000
